In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
# Charger les données prétraitées
X = pd.read_csv("dataset_preprocessed_X.csv")
Y = pd.read_csv("dataset_preprocessed_Y.csv")

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
print("Division des données en ensembles d'entraînement et de test...")
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

print(f"Dimensions de X_train : {X_train.shape}, X_test : {X_test.shape}")
print(f"Dimensions de y_train : {y_train.shape}, y_test : {y_test.shape}")

In [ ]:
def train_lightgbm(X_train, y_train, X_test, y_test):
    """
    Entraîne un modèle LightGBM et évalue ses performances.
    """
    # Définir les hyperparamètres à rechercher
    param_distributions = {
        "learning_rate": np.logspace(-3, 0, 5),  # De 0.001 à 1
        "n_estimators": [50, 100, 200, 500],
        "num_leaves": [31, 50, 100, 150],
        "max_depth": [-1, 10, 20, 30],
        "subsample": [0.7, 0.8, 0.9, 1.0],
        "colsample_bytree": [0.7, 0.8, 0.9, 1.0],
        "reg_alpha": [0.0, 0.1, 1.0],
        "reg_lambda": [0.0, 0.1, 1.0]
    }

    # Initialiser LightGBM
    lgbm = LGBMRegressor(random_state=42)

    # Utiliser RandomizedSearchCV pour rechercher les meilleurs hyperparamètres
    random_search = RandomizedSearchCV(
        estimator=lgbm,
        param_distributions=param_distributions,
        n_iter=100,
        cv=5,
        scoring="r2",
        verbose=2,
        n_jobs=-1,
        random_state=42
    )
    random_search.fit(X_train, y_train.values.ravel())  # y_train.values.ravel() pour une sortie compatible avec LightGBM

    # Meilleur modèle trouvé
    best_model = random_search.best_estimator_

    # Prédictions sur les ensembles d'entraînement et de test
    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)

    # Évaluer les performances
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)

    print("\nMeilleurs hyperparamètres :", random_search.best_params_)
    print("R² Score (train):", train_r2)
    print("R² Score (test):", test_r2)
    print("Mean Squared Error (train):", train_mse)
    print("Mean Squared Error (test):", test_mse)

    return best_model, y_train_pred, y_test_pred


In [ ]:
# Entraîner LightGBM
print("Entraînement du modèle LightGBM...")
best_model, y_train_pred, y_test_pred = train_lightgbm(X_train, y_train, X_test, y_test)

In [ ]:
def plot_results(y_test, y_test_pred, title="Résultats LightGBM"):
    # Graphique des valeurs prédites vs réelles
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_test_pred, alpha=0.7, label="Prédictions")
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="Idéal")
    plt.xlabel("Valeurs réelles (y_test)")
    plt.ylabel("Valeurs prédites (y_pred)")
    plt.title(f"Comparaison entre les valeurs réelles et prédites ({title})")
    plt.legend()
    plt.grid()
    plt.show()

    # Graphique des résidus
    residuals = y_test.values.ravel() - y_test_pred
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test_pred, residuals, alpha=0.7, color="orange")
    plt.axhline(y=0, color="r", linestyle="--", label="Résidus = 0")
    plt.xlabel("Valeurs prédites (y_pred)")
    plt.ylabel("Résidus")
    plt.title(f"Graphique des résidus ({title})")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
# Visualiser les métriques
plot_results(y_test, y_test_pred, title="LightGBM")

In [ ]:
# Affichage des scores R² pour chaque cible (s'il y a plusieurs cibles)
if y_test.shape[1] > 1:
    r2_scores = [r2_score(y_test.iloc[:, i], y_test_pred[:, i]) for i in range(y_test.shape[1])]
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(r2_scores)), r2_scores)
    plt.xlabel("Cibles")
    plt.ylabel("R² Score")
    plt.title("R² Scores pour chaque cible")
    plt.show()